In [3]:
import numpy as np
import pandas as pd
from google.colab import files
uploaded = files.upload()

Saving train.csv to train.csv


In [4]:
train_data = pd.read_csv("train.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.head()
pass_id = test_data.PassengerId

FileNotFoundError: ignored

In [ ]:
train_data.describe()

In [ ]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.head()

In [ ]:
train_data["Cabin"] = train_data["Cabin"].str.replace('\d+', '', regex=True)
cabin_content = []
for index, row in train_data.iterrows():
    if row["Cabin"] not in ["A", "B", "C", "D", "E", "F", "G"]:
        if row["Pclass"] == 1:
            cabin_content.append("C")
        elif row["Pclass"] == 2:
            cabin_content.append("D")
        else:
            cabin_content.append("G")
    else:
        cabin_content.append(row["Cabin"])
train_data["Cabin_letter"] = cabin_content
print(train_data["Cabin_letter"])

In [ ]:
cabin_content = []
for index, row in test_data.iterrows():
    if row["Cabin"] not in ["A", "B", "C", "D", "E", "F", "G"]:
        if row["Pclass"] == 1:
            cabin_content.append("C")
        elif row["Pclass"] == 2:
            cabin_content.append("D")
        else:
            cabin_content.append("G")
    else:
        cabin_content.append(row["Cabin"])
test_data["Cabin_letter"] = cabin_content
print(test_data["Cabin_letter"])

In [ ]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
sex_array = ohe.fit_transform(train_data[["Sex"]]).toarray()
sex_labels = np.array(ohe.categories_).ravel()
sex_data = pd.DataFrame(sex_array, columns = sex_labels)
train_data = pd.concat([train_data, sex_data], axis=1)
train_data = train_data.drop("male", axis="columns")
train_data.rename(columns = {"female":"Gender"}, inplace = True)

ohe2 = OneHotEncoder()
sex_array = ohe2.fit_transform(test_data[["Sex"]]).toarray()
sex_labels = np.array(ohe2.categories_).ravel()
sex_data = pd.DataFrame(sex_array, columns = sex_labels)
test_data = pd.concat([test_data, sex_data], axis=1)
test_data = test_data.drop("male", axis="columns")
test_data.rename(columns = {"female":"Gender"}, inplace = True)
test_data

In [ ]:
from scipy.stats.stats import pearsonr

print("Correlations")
age_data = pd.get_dummies(train_data[["Survived", "Age"]])
age_mean = age_data['Age'].mean()
age_data_drop = age_data.dropna()
corr = pearsonr(age_data_drop["Survived"], age_data_drop["Age"])
print("Raw age: ", corr)

age_data['Age'].fillna(value=age_mean, inplace=True)
corr = pearsonr(age_data["Survived"], age_data["Age"])
print("Filled age: ", corr)

correlated_columns = ["Pclass", "SibSp", "Parch", "Fare", "Gender"]
for name in correlated_columns:
    corr = pearsonr(train_data['Survived'], train_data[name])
    print(name, ": ", corr)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
import statistics as st

features = ["Pclass", "Gender", "Fare", "Parch", "Age", "Embarked"]
age_mean = train_data['Age'].mean()
train_data["Age"].fillna(value=age_mean, inplace=True)
X_all = pd.get_dummies(train_data)
X = pd.get_dummies(train_data[features])
y = train_data["Survived"]


print("Cross-Validation Scores")
pipe_forest = Pipeline([("scaler", StandardScaler()),
      ("model", RandomForestClassifier(n_estimators=100, max_depth=8))])
print("Forest score: ", cross_val_score(pipe_forest, X, y, cv=20, scoring="f1").mean())
pipe_svc = Pipeline([("scaler", StandardScaler()),
      ("model", SVC())])
print("SVC score: ", cross_val_score(pipe_svc, X, y, cv=20, scoring="f1").mean())
pipe_logic = Pipeline([("scaler", StandardScaler()),
      ("model", LogisticRegression(solver='lbfgs', max_iter=200))])
print("Logical regression score: ", cross_val_score(pipe_logic, X, y, cv=20,
                                                    scoring="f1").mean())
pipe_xgb = Pipeline([("scaler", StandardScaler()),
      ("model", XGBClassifier(n_estimators=100, max_depth=8))])
print("XGB score: ", cross_val_score(pipe_xgb, X, y, cv=20, scoring="f1").mean())

print("Test-Split Scores")
model_names = ["forest", "SVC", "logic", "XGB"]
forest_train = []
forest_test = []
sgd_train = []
sgd_test = []
svc_train = []
svc_test = []
logic_train = []
logic_test = []
xgb_train = []
xgb_test = []


for i in range (99):
    X_train, X_test, y_train, y_test=train_test_split(X,y,train_size=0.8,
                                                      random_state=i)
    
    pipe_forest.fit(X_train, y_train)
    forest_train.append(f1_score(pipe_forest.predict(X_train), y_train))
    forest_test.append(f1_score(pipe_forest.predict(X_test), y_test))

    pipe_svc.fit(X_train, y_train)
    svc_train.append(f1_score(pipe_svc.predict(X_train), y_train))
    svc_test.append(f1_score(pipe_svc.predict(X_test), y_test))

    pipe_logic.fit(X_train, y_train)
    logic_train.append(f1_score(pipe_logic.predict(X_train), y_train))
    logic_test.append(f1_score(pipe_logic.predict(X_test), y_test))
    
    pipe_xgb.fit(X_train, y_train)
    xgb_train.append(f1_score(pipe_xgb.predict(X_train), y_train))
    xgb_test.append(f1_score(pipe_xgb.predict(X_test), y_test))

for name in model_names:
    if name == "forest":
        train = forest_train
        test = forest_test
    elif name == "SVC":
        train = svc_train
        test = svc_test
    elif name == "logic":
        train = logic_train
        test = logic_test
    elif name == "XGB":
        train = xgb_train
        test = xgb_test
        
    print(name)
    print("training score")
    print("Min: ", min(train))
    print("Max: ", max(train))
    print("Mean: ", st.mean(train))
    print("testing score")
    print("Min: ", min(test))
    print("Max: ", max(test))
    print("Mean: ", st.mean(test))
    print("")

In [ ]:
# Ensemble time!
from sklearn import model_selection
from sklearn.ensemble import VotingClassifier

fold = model_selection.KFold(n_splits=5, shuffle=True)
estimators = []
estimators.append(("forest", RandomForestClassifier(n_estimators=100, max_depth=8)))
estimators.append(("svc", SVC()))
estimators.append(("xgb", XGBClassifier(n_estimators=100, max_depth=8)))
estimators.append(("logic", LogisticRegression(solver='lbfgs', max_iter=200)))
ensemble = VotingClassifier(estimators)
fin = model_selection.cross_val_score(ensemble, X, y, cv=fold, scoring="f1")
print(fin.mean())
# ensemble.fit(X_train,y_train)
# print(f1_score(ensemble.predict(X_test), y_test))
ensemble.fit(X,y)
print(f1_score(ensemble.predict(X), y))

fare_mean = test_data['Fare'].mean()
test_data["Fare"].fillna(value=fare_mean, inplace=True)
age_mean_test = test_data['Age'].mean()
test_data["Age"].fillna(value=age_mean_test, inplace=True)
test_data = pd.get_dummies(test_data[features])

predictions = ensemble.predict(test_data)
output = pd.DataFrame({'PassengerId': pass_id, 'Survived': predictions})
output.to_csv('submission.csv', index=False)